<a href="https://colab.research.google.com/github/hrishipoola/Valuing_Hologic/blob/main/Valuing_Hologic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valuing Hologic


## Table of Contents
1. Introduction
2. Install & Import Packages
3. Free Cash Flow to the Firm (FCFF) Forecast
<br> FCFF
<br> Forecast
4. Weighted Average Cost of Capital (WACC)
<br> Required ROE
<br> Required ROD
<br> Capital Structure
<br> Calculate WACC
5. Valuation
6. References


## 1. Introduction

Today, we'll value [Hologic (HOLX)](https://www.hologic.com/) using a discounted cash flow model. HOLX develops, manufactures, and supplies diagnostic systems and assays, medical imaging systems, and surgical products focusing on women's health with $3.8B in sales in 2020. It also received approval for a SARS CoV-2 molecular test last year. Despite earnings headwinds in the next few years, HOLX seems undervalued with 26% revenue growth expected in 2021 and below industry-average forward P/E of 14.91, P/S of 4.23, and P/CF of about 11. While HOLX it's seen increased insider selling over the past year, it still has a healthy level of insider holdings (0.6%). Interestingly, in 2021 alone, HOLX has gone on a buying spree, announcing acquisitions of molecular diagnostics company Diagencode biopsy and localization tech company Somatex and completed its acquisition of molecular oncology test provider Biotheranostics. 

Let's value HOLX with a simple 2-stage cash flow model. We'll use [AlphaWave's API](https://www.alphawavedata.com/) to gather the financials and follow along [AlphaWave's valuation example](https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/AlphaWave%20Stock%20Valuation%20using%20Free%20Cash%20Flow%20to%20the%20Firm%20example.ipynb) where they've laid out a 2-stage valuation model. We'll calculate free cash flow to the firm (FCFF), use the FCFF CAGR for the past few years to forecast FCFF for the next 5 years (stage 1), and set an expected constant growth rate in perpetuity thereafter (stage 2).

We'll then discount FCFF at the weighted average cost of capital (WACC). To arrive at WACC, we'll first calculate required return on equity (ROE) using the risk-free rate (10 year Treasury yield), levered beta, and equity risk premium. We'll then calculate required required return on debt (ROD) using interest expense and total debt. Lastly we'll incorporate HOLX's capital structure (mix of equity and debt) to calculate WACC (discount rate). 

Finally, we'll value HOLX share price by discounting FCFF for the first five years (stage 1) and adding the present value of the terminal value at year 6 (stage 2). Using the number of outstanding shares, let's calculate the price per share from our model and compare it to the actual price. 

Our model price of \$117 is significantly higher the actual price of \$74, suggesting HOLX is significantly undervalued. We should keep in mind that the valuation is sensitive to WACC and our growth rate assumptions. Future areas to explore is to test the sensitivity of the valuation against different expected growth rates, applying analyst free cash flow forecasts for the next several years for stage 1 and using a 3-stage model with a gradual decrease in growth rate before a perpetual growth rate in stage 3. 
 

## 2. Install & Import Packages

In [2]:
import pandas as pd
import numpy as np
import requests
from pandas_datareader import data as pdr
import datetime

In [3]:
company_ticker = 'HOLX'

[AlphaWave Licence](https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/LICENSE)

MIT License

Copyright (c) 2020 HDVI

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [4]:
#@title API host and key
#@API host and key
financial_statements_host = "financial-statements.p.rapidapi.com"
financial_statements_key = "e1989dda89mshba802e24464b021p116831jsn330438966071"

stock_analysis_host = "stock-analysis.p.rapidapi.com"
stock_analysis_key = "e1989dda89mshba802e24464b021p116831jsn330438966071"

## 3. FCFF Forecast

We'll take the CAGR of the FCFF for the past 3 years and apply it to FCFF for the next 5 years to generate the FCFF forecast (stage 1). 

For long term expected growth in perpetuity (stage 2), we'll use a very conservative estimate close to GDP growth

### FCFF

There are many ways to calculate FCFF, we'll use the below from Investopedia: 

FCFF = CFO + (IE x (1 - TR) - CAPEX

CFO: cash flow from operations
<br>IE: interest expense
<br>TR: tax rate
<br>CAPEX: capital expenditure

#### Cash Flow Statement

In [5]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/cash-flow"

querystring = {"ticker": company_ticker}

headers = {
    'x-rapidapi-host': financial_statements_host,
    'x-rapidapi-key': financial_statements_key
    }

cash_flow_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Cash Flow Statement DataFrame
cash_flow_df = pd.DataFrame.from_dict(cash_flow_response.json())
cash_flow_df = cash_flow_df.drop('ttm', axis = 1)
cash_flow_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Capital Expenditure,-107600,-105600,-113600,-156400
End Cash Position,540600,666700,601800,701000
Financing Cash Flow,-309200,-404800,-431500,-659900
Free Cash Flow,-99300,627300,535900,740200
Income Tax Paid Supplemental Data,867800,178200,180600,None
Interest Paid Supplemental Data,102400,122100,132500,None
Investing Cash Flow,285800,-195200,-280700,-141600
Issuance of Debt,393000,4034000,2023000,766000
Operating Cash Flow,8300,732900,649500,896600
Repayment of Debt,-529500,-4152300,-2280700,-797500


#### Income Statement

In [6]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/income-statement"

querystring = {"ticker": company_ticker}

headers = {
    'x-rapidapi-host': financial_statements_host,
    'x-rapidapi-key': financial_statements_key
    }

income_statement_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Income Statement DataFrame
income_statement_df = pd.DataFrame.from_dict(income_statement_response.json())
income_statement_df = income_statement_df.drop('ttm', axis = 1)
income_statement_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Basic Average Shares,279811,275105,269413,262727
Cost of Revenue,1437800,1521200,2196400,1548900
Diluted Average Shares,285653,275105,269413,264613
Diluted NI Available to Com Stockholders,755500,-111300,-203600,1115200
EBIT,1383700,-269900,-116900,1118400
Gross Profit,1621000,1696700,1170900,2227500
Interest Expense,153200,148700,140800,116500
Interest Income,3800,6300,4600,4300
Net Income Common Stockholders,755500,-111300,-203600,1115200
Net Income from Continuing & Discontinued Operation,755500,-111300,-203600,1115200


In [7]:
cash_flow_operations = cash_flow_df.loc['Operating Cash Flow'].astype(int) 
interest_expense = income_statement_df.loc['Interest Expense'].astype(int)
tax_rate = income_statement_df.loc['Tax Provision'].astype(int) / income_statement_df.loc['Pretax Income'].astype(int)
capex = cash_flow_df.loc['Capital Expenditure'].astype(int) 

fcff = (cash_flow_operations + (interest_expense * (1 - tax_rate)) - (abs(capex))).astype(int)
fcff = fcff.to_frame().T
fcff

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
0,-5238,666837,647141,869327


### Forecast

In [8]:
# FCFF CAGR
latest_fcff = fcff.iloc[0,-1]
earliest_fcff = fcff.iloc[0,1]
fcff_CAGR = (latest_fcff / earliest_fcff) ** (float(1/(len(fcff.columns)-2)))-1
fcff_CAGR

0.14177818140404663

In [10]:
# Constant expected growth rate 
long_term_growth = fcff_CAGR / 5 # Conservative estimate close to GDP growth for long term perpetual growth 
long_term_growth

0.028355636280809327

In [11]:
# Forecasted FCFF
forecasted_fcff_df = pd.DataFrame(columns=['Year ' + str(i) for i in range(1,7)])
fcff_forecast_lst = []
for i in range(1,7):
    if i != 6:
        fcff_forecast = latest_fcff * (1+fcff_CAGR) ** i
    else:
        fcff_forecast = latest_fcff * (1+fcff_CAGR) ** (i-1) * (1+long_term_growth)
    fcff_forecast_lst.append(int(fcff_forecast))
forecasted_fcff_df.loc[0] = fcff_forecast_lst
forecasted_fcff_df

,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6
0,992578,1133304,1293982,1477440,1686909,1734743


## 4. Weighted Average Cost of Capital (WACC)

### Required ROE

To calclulate required return on equity, we'll apply CAPM:

required ROE = risk-free rate + (levered beta x market risk premium)

#### Balance Sheet

In [12]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/balance-sheet"

querystring = {"ticker": company_ticker}

headers = {
    'x-rapidapi-host': financial_statements_host,
    'x-rapidapi-key': financial_statements_key
    }

balance_sheet_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Balance Sheet DataFrame
balance_sheet_df = pd.DataFrame.from_dict(balance_sheet_response.json())
balance_sheet_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Capital Lease Obligations,58400,58100,54700,108400
Common Stock Equity,2784700,2428800,2115700,2705200
Invested Capital,6107600,5733100,5170700,5744000
Net Debt,2782300,2637600,2453200,2337800
Net Tangible Assets,-3140400,-2503000,-1907800,-1260200
Ordinary Shares Number,275293,270088,267685,257498
Share Issued,287853,289900,292323,295107
Tangible Book Value,-3140400,-2503000,-1907800,-1260200
Total Assets,7979600,7230900,6442100,7195800
Total Capitalization,4956800,5133400,4899300,5419100


#### Risk-Free Rate

In [13]:
# For risk free rate, we'll use the latest 10-year U.S. Treasury Yield '^TNX'
risk_free_rate = (pdr.get_data_yahoo('^TNX')['Adj Close'].iloc[-1]) / 100
risk_free_rate

0.016790000200271608

#### Levered Beta

##### Key Stats

In [14]:
url = "https://stock-analysis.p.rapidapi.com/api/v1/resources/key-stats"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': stock_analysis_host,
    'x-rapidapi-key': stock_analysis_key
    }

key_stats_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Key Statistics DataFrame
key_stats_df = pd.DataFrame.from_dict(key_stats_response.json())
key_stats_df = key_stats_df.transpose()
key_stats_df

,Value
% held by insiders,0.59%
% held by institutions,97.10%
200-day moving average,72.30
5-year average dividend yield,0
50-day moving average,74.06
52-week change,126.63%
52-week high,85.00
52-week low,32.55
Avg vol (10-day),1.51M
Avg vol (3-month),2.02M


In [15]:
# Beta
beta = float(key_stats_df.loc['Beta (5Y monthly)'])
beta

1.2

In [16]:
# Total Debt
total_debt = balance_sheet_df.loc['Total Debt']
total_debt_df = total_debt.to_frame().transpose()
total_debt_str = total_debt_df.values[0][-1:]
total_debt_int = int(total_debt_str)

In [17]:
# Market Value of Equity
market_cap_str = key_stats_df.loc[r'Market cap (intra-day) '][0]

market_cap_lst = market_cap_str.split('.')
if market_cap_str[len(market_cap_str)-1] == 'T':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('T',(9-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'B':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('B',(6-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))

market_cap_int

19150000

In [18]:
# Avg Tax Rate
avg_tax_rate = sum(tax_rate) / len(tax_rate)
avg_tax_rate

0.30541890879797007

In [19]:
# Levered beta 
levered_beta = beta * (1+(1-avg_tax_rate)*(total_debt_int / market_cap_int))
levered_beta

1.3369808215288372

### Equity Risk Premium

We'll use implied equity risk premium from professor [Aswath Damodaran](http://pages.stern.nyu.edu/~adamodar/) 

In [20]:
# Equity risk premium
equity_risk_premium = 0.0463

### Required ROE

In [21]:
# Required ROE
required_roe = risk_free_rate + (levered_beta * equity_risk_premium)
required_roe

0.07869221223705677

### Required ROD

In [22]:
# Interest Expense
interest_expense = income_statement_df.loc['Interest Expense']
interest_expense_df = interest_expense.to_frame().transpose()
interest_expense_str = interest_expense_df.values[0][-1:]
interest_expense_int = int(interest_expense_str)

# Total debt above

# Required Return on Debt
required_return_debt = interest_expense_int / total_debt_int
required_return_debt

0.03701703101169293

### Capital Structure

The final inputs for calculating WACC deal with the firm's capital structure, which is a company's mix of equity and debt. As previously noted, WACC is a blend of a company’s equity and debt cost of capital based on the company's equity and debt capital ratio.

In most cases, you can use the book value of debt from a company’s latest balance sheet as an approximation for market value of debt. Unlike equity, the market value of debt usually does not deviate too far from the book value.

In [23]:
# Market Value of Debt
net_debt = balance_sheet_df.loc['Net Debt']
net_debt_df = net_debt.to_frame().T
net_debt_str = net_debt_df.values[0][-1:]
net_debt_int = int(net_debt_str)
net_debt_int

2337800

In [24]:
# Company Value
company_value = market_cap_int + net_debt_int
company_value

21487800

### Calculate WACC


In [25]:
# https://www.investopedia.com/ask/answers/063014/what-formula-calculating-weighted-average-cost-capital-wacc.asp
WACC = ((market_cap_int / company_value) * required_roe) + ((net_debt_int / company_value) * (required_return_debt * (1 - avg_tax_rate)))
WACC

0.07292807133024737

## 5. Valuation
We need to discount all the forecasted cash flows to get the Firm Value. First, discount the forecasted FCFF for each of the next five years (Stage 1). Then, the present value of the terminal value of the firm using the forecasted FCFF in Year 6 is added to this amount (Stage 2).

In [26]:
# Equity Value Calculation
discounted_fcff_lst = []
for year in range(0,5):
    discounted_fcff = forecasted_fcff_df.iloc[0, year] / (1 + WACC) ** (year+1)
    discounted_fcff_lst.append(int(discounted_fcff))

terminal_value = forecasted_fcff_df.iloc[0,5] / (WACC-long_term_growth)
PV_terminal_value = int(terminal_value / (1+WACC) **5)
enterprise_value = sum(discounted_fcff_lst) + PV_terminal_value
equity_value = enterprise_value - net_debt_int
equity_value

30293315

In [27]:
# Total Shares Outstanding
shares_outstanding_str = key_stats_df.loc['Shares outstanding '][0]

shares_outstanding_lst = shares_outstanding_str.split('.')
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'B':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('B',(6-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'M':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('M',(3-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))

shares_outstanding_int

257660

In [28]:
# Two-stage FCFF Valuation Model Stock Price Estimate
stock_price = equity_value / shares_outstanding_int
stock_price = '${:,.2f}'.format(stock_price)
print("Model Stock Price = %s"%(stock_price))

# Actual Stock Price
actual_stock_price = market_cap_int / shares_outstanding_int
actual_stock_price = '${:,.2f}'.format(actual_stock_price)
print("Actual Stock Price = %s"%(actual_stock_price))

Model Stock Price = $117.57
Actual Stock Price = $74.32


## 6. References

https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/AlphaWave%20Stock%20Valuation%20using%20Free%20Cash%20Flow%20to%20the%20Firm%20example.ipynb

https://investors.hologic.com/financials/financial-reports/default.aspx

https://www.investopedia.com/terms/f/freecashflowfirm.asp

https://www.investopedia.com/ask/answers/063014/what-formula-calculating-weighted-average-cost-capital-wacc.asp

https://simplywall.st/stocks/us/healthcare/nasdaq-holx/hologic/news/does-this-valuation-of-hologic-inc-nasdaqholx-imply-investors-are-overpaying

https://finance.yahoo.com/news/value-investors-buy-hologic-holx-165004814.html

http://pages.stern.nyu.edu/~adamodar/



